In [73]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M5','M15','H1','H3','D1','W1'])
Query.head()

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['A']        = Raw['a']
    Src['Z']        = Raw['z']
    Src['Symbol']   = Raw['symbol']
    Src['TF']       = Raw['tf']
    
    Src['Datetime'] = Raw['datetime']
    Src['Date']     = Raw['datetime'].dt.date
    Src['Time']     = Raw['datetime'].dt.time
    
    Src['Ticks']    = Raw['tick_volume']
    Src['Volume']   = Raw['real_volume']
    Src['Price']    = Raw['close']
    
    Src['Open']     = Raw['open']
    Src['High']     = Raw['high']
    Src['Low']      = Raw['low']
    Src['Close']    = Raw['close']

    Src['Direct']       = (Src['Close'] - Src['Open']).apply(lambda x: -1 if (x <  0) else  +1)
    Src['Bull Sign']    =                Src['Direct'].apply(lambda x:  x if (x >= 0) else nan)
    Src['Bear Sign']    =                Src['Direct'].apply(lambda x:  x if (x <  0) else nan)
    
    Src['Change']       =  Src['Close'] - Src['Open']
    Src['HL']           =  Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Raw=Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Change Abs']  = Src['Change'].abs()
    Calc['Change Pos']  = Src['Change'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Change Neg']  = Src['Change'].apply(lambda x: x if (x <  0) else nan)
    Calc['HL Pos']      = Src['Bull Sign'] * Src['HL']
    Calc['HL Neg']      = Src['Bear Sign'] * Src['HL']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [78]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change']].describe().round(0).astype(int)

Change                                         
            count mean   std    min   25%  50%   75%   max
Symbol TF                                                 
WIN$   M5   99999    0   142  -2670   -67    0    68  3070
       M15  46109   -1   272  -2962  -129    0   128  5312
       H1   11866   -3   540  -4266  -261    5   268  5561
       H3    4408   -7   891  -5468  -414    6   411  5697
       D1    1246  -25  1711  -7641 -1083   60  1019  5768
       W1     262  -29  3510 -11582 -2223 -120  2355  9576

In [79]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change Abs']].describe().round(0).astype(int)

Change Abs                                         
                count  mean   std min   25%   50%   75%    max
Symbol TF                                                     
WIN$   M5       99999    97   103   0    30    68   129   3070
       M15      46109   188   197   0    57   128   251   5312
       H1       11866   378   385   0   114   265   508   5561
       H3        4408   620   639   0   160   412   872   5697
       D1        1246  1327  1080   0   504  1052  1848   7641
       W1         262  2731  2198   0  1022  2310  3904  11582

In [80]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change Pos']].describe().round(0).astype(int)

Change Pos                                        
                count  mean   std min   25%   50%   75%   max
Symbol TF                                                    
WIN$   M5       51539    94   103   0    28    65   125  3070
       M15      23421   184   199   0    54   126   248  5312
       H1        6013   371   382   0   110   264   498  5561
       H3        2254   599   613   0   156   397   855  5697
       D1         644  1259  1021   0   453   982  1829  5768
       W1         124  2855  2141   0  1229  2506  3930  9576

In [81]:
Calc.groupby(['Symbol','TF'], sort=0)[['Change Neg']].describe().round(0).astype(int)

Change Neg                                        
                count  mean   std    min   25%   50%  75% max
Symbol TF                                                    
WIN$   M5       48460  -100   102  -2670  -133   -70  -33  -5
       M15      22688  -192   196  -2962  -255  -132  -60  -5
       H1        5853  -386   388  -4266  -517  -265 -118  -5
       H3        2154  -642   664  -5468  -888  -434 -164  -5
       D1         602 -1399  1135  -7641 -1872 -1115 -575  -6
       W1         138 -2620  2250 -11582 -3896 -2048 -910 -45

In [82]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL']].describe().round(0).astype(int)

HL                                           
            count  mean   std   min   25%   50%   75%    max
Symbol TF                                                   
WIN$   M5   99999   201   131     0   115   168   250   3565
       M15  46109   391   251     0   221   330   494   6275
       H1   11866   772   482    75   438   659   988   7753
       H3    4408  1237   823    75   640  1086  1663   7753
       D1    1246  2547  1156   593  1703  2322  3130   8487
       W1     262  5677  2340  1708  4002  5146  7076  17818

In [83]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)

HL Pos                                           
            count  mean   std   min   25%   50%   75%    max
Symbol TF                                                   
WIN$   M5   51539   199   132     0   113   165   247   3565
       M15  23421   386   255     0   217   325   484   6275
       H1    6013   761   484    75   433   650   975   7753
       H3    2254  1206   806    75   620  1068  1629   7753
       D1     644  2459  1090   593  1660  2258  3035   7864
       W1     124  5636  2323  1708  3779  5163  7164  12978

In [84]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

HL Neg                                           
            count  mean   std    min   25%   50%   75%   max
Symbol TF                                                   
WIN$   M5   48460  -204   130  -2722  -254  -171  -118   -10
       M15  22688  -397   247  -3209  -502  -335  -225   -45
       H1    5853  -783   480  -5016 -1002  -669  -444  -101
       H3    2154 -1268   839  -6485 -1686 -1109  -663  -104
       D1     602 -2640  1217  -8487 -3264 -2354 -1771  -655
       W1     138 -5714  2364 -17818 -7072 -5120 -4034 -2352

In [85]:
raise Exception('STOP')

Exception: STOP

# Chart

In [ ]:
for SYMBOL, Asset in Src.groupby('Symbol', sort=0):

    plt.title(f'{SYMBOL} - Candle Size Distribuition', fontweight='bold')

    for TF, Data in Asset.groupby('TF', sort=0): 
        # sns.kdeplot(Data['Change'], label=TF)
        # sns.histplot(Data['Change'], label=TF, stat='count', bins=100, element='step', fill=False)
        plt.hist(Data['Change'], bins=100, label=TF, histtype='step')


    plt.ylabel('Frequency')
    plt.xlabel('Candle Size')

    plt.yscale('log', base=10)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.ScalarFormatter())

    plt.legend()
    plt.tight_layout()
    plt.show()

# Snippets

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['Direct']       = (Src['Close'] - Src['Open']).apply(lambda x: -1 if (x <  0) else  +1)
    Src['Bull Sign']    =                Src['Direct'].apply(lambda x:  x if (x >= 0) else nan)
    Src['Bear Sign']    =                Src['Direct'].apply(lambda x:  x if (x <  0) else nan)
    
    Src['Change']       =  Src['Close'] - Src['Open']
    Src['HL']           =  Src['High']  - Src['Low']
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Change Abs']  = Src['Change'].abs()
    Calc['Change Pos']  = Src['Change'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Change Neg']  = Src['Change'].apply(lambda x: x if (x <  0) else nan)
    Calc['HL Pos']      = Src['Bull Sign'] * Src['HL']
    Calc['HL Neg']      = Src['Bear Sign'] * Src['HL']
    return Calc
  
  
Calc.groupby(['Symbol','TF'], sort=0)[['HL']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change Abs']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Change Neg']].describe().round(0).astype(int)